In [65]:
import requests as rq
import pymongo as pm
import ujson as js
import base64 as b64
from datetime import datetime as dt
from datetime import timedelta as td

from utils.email import send_email

In [75]:
class TokenRefresher:
    mongo_login_path = '../settings/mongo_login.json'
    key_path = '../settings/esi_key.json'
    data = {'grant_type': 'refresh_token'}
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Host': 'login.eveonline.com'
    }
    url = 'https://login.eveonline.com/v2/oauth/token'
    collection = 'eve_characters'
    name = 'ESI OAUTH2 Refresh Token'
    
    def __init__(self):
        self.load_key()
        
    def _email(self, error_type, error_body):
        send_email(error_type, {
            'process':self.name,
            'time':datetime.now().isoformat(' '),
            'note':error_body
        })
    
    def connect_mongo(self):
        with open(self.mongo_login_path) as file:
            mongo_settings = js.load(file)
            self.db = mongo_settings['authSource']
            self.conn = pm.MongoClient(**mongo_settings)
            
    def load_key(self):
        with open(self.key_path) as file:
            self.headers['Authorization'] = 'Basic {code}'.format(**{
                'code': b64.b64encode(bytes(
                    '{client_id}:{secret_key}'.format(**js.load(file)), 'utf-8'
                )).decode('utf-8')
            })
    
    def run_process(self):
        try:
            self.connect_mongo()
            self.refresh_tokens()
            self.conn.close()
        except Exception as e:
            send_email('fail', {
                'process':self.name,
                'time':dt.now().isoformat(' '),
                'note':e
            })
    
    def refresh_tokens(self):
        for char in self.conn[self.db][self.collection].find({'tokens.refresh_token': {'$exists': True}}):
            post = rq.post(
                self.url,
                data = {
                    **self.data,
                    'refresh_token': char['tokens']['refresh_token']
                },
                headers = self.headers
            )

            post_content = js.loads(post.content)
            receive_time = dt.strptime(
                post.headers['Date'],
                '%a, %d %b %Y %H:%M:%S %Z'
            )

            self.conn[self.db][self.collection].update_one(
                {'_id': char['_id']},
                {'$set': {
                    'tokens.access_token': post_content['access_token'],
                    'tokens.token_type': post_content['token_type'],
                    'tokens.expires_on': receive_time + td(seconds = post_content['expires_in']),
                    'tokens.last_set': receive_time
                }}
            )

In [76]:
refresh = TokenRefresher()
refresh.run_process()